In [ ]:
import os
import shutil

# Define paths
base_path = "../data/tracking/train"
train_img_dir = "../data//working/MOT17/train/images"
train_label_dir = "../data/working/MOT17/train/labels"

# Clear existing directories
shutil.rmtree(train_img_dir, ignore_errors=True)
shutil.rmtree(train_label_dir, ignore_errors=True)
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)

# Copy with PROPER FILENAMES
for seq in os.listdir(base_path):
    seq_path = os.path.join(base_path, seq)
    img_folder = os.path.join(seq_path, "img1")
    label_folder = os.path.join(seq_path, "labels")  # Your converted labels

    if os.path.exists(img_folder):
        for img_name in os.listdir(img_folder):
            if img_name.endswith(".jpg"):
                # New filename: Sequence + Original name
                new_base = f"{seq}_{os.path.splitext(img_name)[0]}"

                # Copy image
                src_img = os.path.join(img_folder, img_name)
                dest_img = os.path.join(train_img_dir, f"{new_base}.jpg")
                shutil.copy2(src_img, dest_img)

                # Copy corresponding label
                src_label = os.path.join(label_folder, img_name.replace(".jpg", ".txt"))
                dest_label = os.path.join(train_label_dir, f"{new_base}.txt")

                if os.path.exists(src_label):
                    shutil.copy2(src_label, dest_label)

print("✅ Images & labels copied with matching sequence prefixes!")

In [ ]:
import os

def get_image_dimensions(seqinfo_path):
    """Reads seqinfo.ini to extract image width and height."""
    img_width, img_height = None, None
    if os.path.exists(seqinfo_path):
        with open(seqinfo_path, "r") as f:
            for line in f:
                if line.startswith("imWidth"):
                    img_width = int(line.strip().split("=")[1])
                elif line.startswith("imHeight"):
                    img_height = int(line.strip().split("=")[1])
    return img_width, img_height

def convert_mot_to_yolo(mot_base_path, output_base_path):
    os.makedirs(output_base_path, exist_ok=True)

    for sequence in os.listdir(mot_base_path):
        seq_path = os.path.join(mot_base_path, sequence)
        seqinfo_path = os.path.join(seq_path, "seqinfo.ini")
        img_width, img_height = get_image_dimensions(seqinfo_path)

        # Get image files to match frame IDs
        img_folder = os.path.join(seq_path, "img1")
        img_files = sorted([f for f in os.listdir(img_folder) if f.endswith(".jpg")])

        # Create empty labels for all frames
        for img_file in img_files:
            frame_id = int(os.path.splitext(img_file)[0])
            label_file = f"{sequence}_{frame_id:06d}.txt"  # Match image naming
            open(os.path.join(output_base_path, label_file), 'a').close()

        # Process MOT annotations
        mot_path = os.path.join(seq_path, "gt", "gt.txt")
        if os.path.exists(mot_path):
            with open(mot_path, "r") as f:
                for line in f:
                    try:
                      frame_id, track_id, x, y, w, h, conf, class_id, visibility = map(float, line.strip().split(','))

                      # Convert to YOLO format (normalized)
                      x_center = (x + w / 2) / img_width
                      y_center = (y + h / 2) / img_height
                      w = w / img_width
                      h = h / img_height

                      yolo_line = f"{int(class_id)} {x_center} {y_center} {w} {h}\n"
                      frame_txt = f"{int(frame_id):06d}.txt"
                      frame_id = int(float(line.split(',')[0]))
                      label_file = f"{sequence}_{frame_id:06d}.txt"
                      label_path = os.path.join(output_base_path, label_file)

                      # Append to existing label file
                      with open(label_path, "a") as label_file:
                          label_file.write(yolo_line)
                    except Exception as e:
                        print(f"Error processing line in {sequence}: {line} -> {e}")

        print(f"✅ Conversion complete for sequence {sequence}. Labels saved in {output_base_path}")


# Example Usage
convert_mot_to_yolo(
    mot_base_path="../data/tracking/train",
    output_base_path="../data/working/MOT17/train/labels"
)

In [ ]:
import os
import shutil
import random

val_img_dir = "../data/working/MOT17/val/images"
val_label_dir = "../data/working/MOT17/val/labels"

# Create validation directories
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# List all images
image_files = sorted(os.listdir(train_img_dir))
random.shuffle(image_files)  # Shuffle dataset

# Define split ratio (20% validation)
val_count = int(len(image_files) * 0.2)

# Move images & labels to val/
for img_name in image_files[:val_count]:
    img_path = os.path.join(train_img_dir, img_name)
    label_name = img_name.replace(".jpg", ".txt")
    label_path = os.path.join(train_label_dir, label_name)

    # Move to validation set
    shutil.move(img_path, os.path.join(val_img_dir, img_name))
    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(val_label_dir, label_name))

print(f"✅ Moved {val_count} images & labels to validation set!")

In [ ]:
import os

def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        with open(file_path, "r") as f:
            lines = f.readlines()

        # Change class ID from 1 to 0
        updated_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts[0] == "1":
                parts[0] = "0"
                updated_lines.append(" ".join(parts) + "\n")

        # Write updated labels
        with open(file_path, "w") as f:
            f.writelines(updated_lines)

# Update train and val labels
update_labels("../data/working/MOT17/train/labels")
update_labels("../data/working/MOT17/val/labels")

print("✅ Updated class indices to 0 in all labels")

In [ ]:
import yaml

data_yaml = {
    "train": "../data/working/MOT17/train/images",
    "val": "../data/working/MOT17/val/images",
    "nc": 1,
    "names": ["person"],  # Define class names
    "train_labels": "../data/working/MOT17/train/labels",  # Add labels path
    "val_labels": "../data/working/MOT17/val/labels"       # Add labels path
}

yaml_path = "../data/working/data.yaml"

# Save to YAML file
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)


print("done")

In [ ]:
# Check training set
train_images = set([f.replace(".jpg", "") for f in os.listdir("../data/working/MOT17/train/images")])
train_labels = set([f.replace(".txt", "") for f in os.listdir("../data/working/MOT17/train/labels")])
print("Missing training labels:", len(train_images - train_labels))

# Check validation set
val_images = set([f.replace(".jpg", "") for f in os.listdir("../data/working/MOT17/val/images")])
val_labels = set([f.replace(".txt", "") for f in os.listdir("../data/working/MOT17/val/labels")])
print("Missing validation labels:", len(val_images - val_labels))

In [ ]:
import yaml
from sklearn.model_selection import KFold

# Path to COPIED IMAGES (not the original input)
train_img_dir = "../data/working/MOT17/train/images"

# Collect all image paths in working directory
image_paths = [
    os.path.join(train_img_dir, img)
    for img in os.listdir(train_img_dir)
    if img.endswith(".jpg")
]

k = 3  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(image_paths)):
    # Write train/val .txt files with ABSOLUTE PATHS
    with open(f"../data/working/MOT17/fold_{fold}_train.txt", "w") as f:
        f.write("\n".join([image_paths[i] for i in train_idx]))

    with open(f"../data/working/MOT17/fold_{fold}_val.txt", "w") as f:
        f.write("\n".join([image_paths[i] for i in val_idx]))

    # Create YAML
    data_yaml = {
        "train": f"../data/working/MOT17/fold_{fold}_train.txt",
        "val": f"../data/working/MOT17/fold_{fold}_val.txt",
        "nc": 1,
        "names": ["person"]
    }
    with open(f"../data/working/data_fold_{fold}.yaml", "w") as f:
        yaml.dump(data_yaml, f)

# Check if files exist
for fold in range(k):
    train_txt = f"../data/working/MOT17/fold_{fold}_train.txt"
    val_txt = f"../data/working/MOT17/fold_{fold}_val.txt"

    if not os.path.exists(train_txt):
        print(f"❌ Missing: {train_txt}")
    if not os.path.exists(val_txt):
        print(f"❌ Missing: {val_txt}")

# Sample first 3 paths from fold 1
with open("../data/working/MOT17/fold_1_train.txt", "r") as f:
    sample_paths = f.readlines()[:3]
print("Sample paths:", sample_paths)

In [ ]:
from os.path import exists
from sklearn.model_selection import ParameterGrid, ParameterSampler
from ultralytics import YOLO
import torch

# Choose search method (Grid or Random)
search_method = "grid"  # or "random"

# Grid Search Parameters
grid_params = {
    'lr0': [0.01, 0.001],
    'momentum': [0.9, 0.95],
    'weight_decay': [0.0005, 0.0001]
}

# Random Search Parameters
random_params = {
    'lr0': [0.01, 0.005, 0.001],
    'momentum': [0.85, 0.9, 0.95],
    'weight_decay': [0.0005, 0.0002, 0.0001]
}
n_iter = 10

param_iter = ParameterGrid(grid_params) if search_method == "grid" else ParameterSampler(random_params, n_iter)

best_map = 0
best_params = None

for params in param_iter:
    print(f"Training with: {params}")
    fold_metrics = []

    for fold in range(k):
    torch.cuda.empty_cache()

    # Load fresh model for each fold
    model = YOLO("last.pt")

    results = model.train(
        data=f"../data/working/data_fold_{fold}.yaml",
        epochs=1,
        batch=1,
        imgsz=1080,
        workers=4,
        device="cuda",
        save_period=1,
        project="kfold_search",
        name=f"fold_{fold}_lr{params['lr0']}",
        augment=True,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        exist_ok=True,
        # plots=True,  # Enable detailed plots
        **params
    )
    # results = model.train(resume=True)

    fold_metrics.append(results.results_dict['metrics/mAP50-95(B)'])  # Use appropriate metric
    print(f"fold_metrics {fold_metrics}")

    avg_map = sum(fold_metrics) / len(fold_metrics)
    if avg_map > best_map:
        best_map = avg_map
        best_params = params
        print(f"best map & params: {best_map} & {best_params}")

print(f"Best Params: {best_params} | mAP: {best_map}")

In [ ]:
final_model = YOLO("../data/working/runs/detect/train/weights/last.pt")

# Training with best params
final_model.train(
    data="../data/working/data.yaml",
    epochs=3,
    batch=1,
    imgsz=1080,
    workers=4,
    device="cuda",
    save_period=1,
    project="../data/runs/detect",
    name="train",
    **best_params
)

In [ ]:
from ultralytics import YOLO
import torch

model = YOLO("/kaggle/input/last/pytorch/default/1/last.pt")

# Train with aggressive augmentations
results = model.train(
    data='../data/working/data.yaml',
    epochs=4,
    imgsz=1088,  # Match original resolution
    batch=1,
    device=0,
    workers=4,
    augment=True,
    fliplr=0.5,
    mosaic=0.75,
    mixup=0.5,
    degrees=15,
    shear=5,
    perspective=0.0005,
    copy_paste=0.5,
    hsv_h=0.3,
    hsv_s=0.5,
    hsv_v=0.3,
    lr0=0.001,
    lrf=0.1,
    momentum=0.9,
    weight_decay=0.0005,
    save_period=1,
    project="../data/working/runs/detect",
    name="train"
    # plots=True
)